In [1]:
# необходимо чтобы следующие библиотеки были установлены

import cv2 as cv2
import librosa
import numpy as np
import tensorflow as tf


In [4]:
#пути к файлам модели
#file_model - файл модели
#file_weigth - веса модели

file_model  = '/home/123/PycharmProjects/sssoft24.com/audio.h5'
file_weigth = '/home/123/PycharmProjects/sssoft24.com/audio_weight.h5'


def loadmodel(file_model, file_weigth):
    model = tf.keras.models.Model()
    model = tf.keras.models.load_model(file_model)
    model.load_weights(file_weigth)
    return model

model = loadmodel(file_model, file_weigth)
    

In [5]:
classes = {'fon': 0, 'micout': 1, 'noise': 2, 'psk100': 3, 'psk50': 4, 'voice': 5}

def get_class_name(classes, predict_class):
  for p in classes:
    if classes[p] == predict_class:
      return(p) 


def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

def mel_to_img(mel):
    
    pic = np.array(mel)
    img = np.zeros((pic.shape[0], pic.shape[1], 3))
    np.place(img[ :, :, 0], pic[ :, :], pic[ :, :]) 
    np.place(img[ :, :, 1], pic[ :, :], pic[ :, :]) 
    np.place(img[ :, :, 2], pic[ :, :], pic[ :, :]) 
    return img


def convert_mel(freq, sr=22050, n_fft=8192):
  mel = librosa.feature.melspectrogram(freq,sr=sr, n_fft=8192)  
  mel = librosa.power_to_db(mel)
  mel = scale_minmax(mel, 0, 255).astype(np.uint8)
  mel = np.flip(mel, axis=0) 
  
  return mel

In [10]:
def prepare_audio(file_name):
    y, sr = librosa.load(file_name)    
    mel = convert_mel(y, sr=sr)
    mel = cv2.resize(mel, (360,192), interpolation=cv2.INTER_AREA)
    mel = mel_to_img(mel)    
    mel = mel/255
    mel_np = np.expand_dims(mel,0)
    return mel_np
    

In [11]:
def predict(audio_file):    
    img = prepare_audio(audio_file)    
    predict = model.predict(img)
    answer = get_class_name(classes, np.argmax(predict))
    return print('Предсказанный класс: {}'.format(answer))

In [12]:
# в переменную аудио записывается путь до аудиофайла .wav 
audio = '/home/123/PycharmProjects/sssoft24.com/1226 obuch17052021.wav'

In [13]:
# при вызове функции predict передаем параметр пути аудиофайла, в ответ возвращается предикт нейронной сети

predict(audio)

Предсказанный класс: voice
